[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/05-local-execution.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/05-local-execution.ipynb)

# Local Dynamic Routes

## Fully local Semantic Router with `llama.cpp` and HuggingFace Encoder

There are many reasons users might choose to roll their own LLMs rather than use a third-party service. Whether it's due to cost, privacy or compliance, Semantic Router supports the use of "local" LLMs through `llama.cpp`.

Using `llama.cpp` also enables the use of quantized GGUF models, reducing the memory footprint of deployed models, allowing even 13-billion parameter models to run with hardware acceleration on an Apple M1 Pro chip.

Below is an example of using semantic router with **Mistral-7B-Instruct**, quantized i.

## Installing the library

> Note: if you require hardware acceleration via BLAS, CUDA, Metal, etc. please refer to the [abetlen/llama-cpp-python](https://github.com/abetlen/llama-cpp-python#installation-with-specific-hardware-acceleration-blas-cuda-metal-etc) repository README.md

In [3]:
%pip install -qU "semantic-router[local]==0.0.17"

Note: you may need to restart the kernel to use updated packages.


(NO ES EL CASO) If you're running on Apple silicon you can run the following to run with Metal hardware acceleration:

In [1]:
# !CMAKE_ARGS="-DLLAMA_METAL=on"

PARA INSTALAR LLAMA-CPP-PYTHON CON SOPORTE GPU (CUDA) EJECUTAR:

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Sep__8_19:17:24_PDT_2023
Cuda compilation tools, release 12.3, V12.3.52
Build cuda_12.3.r12.3/compiler.33281558_0


Esto me carga el modelo en GPU pero luego me crashea.

In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.3.1 from /home/javier/miniconda3/envs/llama-cpp/lib/python3.10/site-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 43.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ...   Running command pip subprocess to install build dependencies
    Using cached scikit_build_core-0.7.1-py3-none-any.whl.metadata (19 kB)
    Using cached exceptiongroup-1.2.0-py3-none-any.whl.metadata (6.6 kB)
    Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
    Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
  Using cached scikit_build_core-0.7.1-py3-none-any.whl (136 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
done
  Getting requirements to build wheel ...   Running command

In [1]:
%pip uninstall -y llama-cpp-python

Found existing installation: llama_cpp_python 0.2.31
Uninstalling llama_cpp_python-0.2.31:
  Successfully uninstalled llama_cpp_python-0.2.31
Note: you may need to restart the kernel to use updated packages.


In [2]:
#With NVidia CUDA acceleration
!CMAKE_ARGS="-DLLAMA_CUBLAS=on"  pip install llama-cpp-python 

  Using cached llama_cpp_python-0.2.31-cp310-cp310-manylinux_2_35_x86_64.whl
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Como tengo problemas de incompatibilidad de xformers con torch (debería ser 2.1.0) en lugar de arriesgarme a cambiarlo que y se me rompan otras cosas voy a crear un entorno nuevo con conda y voy a instalar todo desde cero.


```bash
conda create -n llama-cpp python=3.10
conda activate llama-cpp
conda install pytorch==2.1.0 -c pytorch
conda install pytorch==2.1.0 torchvision torchaudio cudatoolkit=12.2 -c pytorch #la versión de CUDA del sistema es 12.2 (se ve con nvidia-smi)

#verificar
python -c "import torch; print(torch.__version__)"
#2.1.0



In [1]:
%%bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp

Cloning into 'llama.cpp'...
Updating files: 100% (398/398), done.


In [2]:
%%bash
cd llama.cpp
make LLAMA_CUBLAS=1

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi
I NVCCFLAGS: -

common/common.cpp: In function ‘bool gpt_params_parse_ex(int, char**, gpt_params&)’:
common/common.cpp:223:23: warning: cast from type ‘const char*’ to type ‘char*’ casts away qualifiers [-Wcast-qual]
  223 |             file.read((char *)params.prompt.data(), size);
      |                       ^~~~~~~~~~~~~~~~~~~~~~~~~~~~


g++ -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -c common/sampling.cpp -o sampling.o
g++ -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -c common/grammar-parser.cpp -o grammar-parser.o
g++ -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS 

examples/perplexity/perplexity.cpp: In function ‘bool deserialize_string(std::istream&, std::string&)’:
examples/perplexity/perplexity.cpp:1038:22: warning: cast from type ‘const char*’ to type ‘char*’ casts away qualifiers [-Wcast-qual]
 1038 |         if (!in.read((char *)str.data(), size).fail()) return true;
      |                      ^~~~~~~~~~~~~~~~~~


g++ -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi examples/imatrix/imatrix.cpp ggml.o llama.o common.o sampling.o grammar-parser.o build-info.o ggml-cuda.o ggml-alloc.o ggml-backend.o ggml-quants.o -o imatrix -lcuda -lcublas -lculibos -lcudart -lcublasLt -lpthread -ldl -lrt -L/usr/local/cuda/lib64 -L/opt/cuda/lib64 -L/targets/x86_64-linux/lib -L/usr/local/cuda/targets/aarch64-linux/lib -L/usr/lib/wsl/lib 
g++ -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_CUBLAS -I/usr/local/cuda/include -I/opt/cuda/include -I/targets/x86_64-linux/include -I/usr/local/cuda/targets/aarch64-linux/include  -std=c++11 -fPIC -

In [9]:
%pip uninstall -y llama-cpp-python

Found existing installation: llama_cpp_python 0.2.31
Uninstalling llama_cpp_python-0.2.31:
  Successfully uninstalled llama_cpp_python-0.2.31
Note: you may need to restart the kernel to use updated packages.


In [2]:
# poner variables de entorno en cuaderno jupyter
%env CUDA_HOME=/usr/local/cuda-12/

env: CUDA_HOME=/usr/local/cuda-12/


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_COMPILER=/usr/local/cuda-12/bin/nvcc " FORCE_CMAKE=1 pip install --break-system-packages  --upgrade --force-reinstall llama-cpp-python --no-cache-dir --verbose;

Using pip 23.3.1 from /home/javier/miniconda3/envs/llama-cpp/lib/python3.10/site-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ...   Running command pip subprocess to install build dependencies
    Using cached scikit_build_core-0.7.1-py3-none-any.whl.metadata (19 kB)
    Using cached exceptiongroup-1.2.0-py3-none-any.whl.metadata (6.6 kB)
    Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
    Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
  Using cached scikit_build_core-0.7.1-py3-none-any.whl (136 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl (16 kB)
done
  Getting requirements to build wheel ...   Running command

In [7]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on"  pip install llama-cpp-python 
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

  Using cached llama_cpp_python-0.2.31-cp310-cp310-manylinux_2_35_x86_64.whl


## Download the Mistral 7B Instruct 4-bit GGUF files

We will be using Mistral 7B Instruct, quantized as a 4-bit GGUF file, a good balance between performance and ability to deploy on consumer hardware

In [1]:
! curl -L "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf?download=true" -o ./mistral-7b-instruct-v0.2.Q5_K_M.gguf
! ls mistral-7b-instruct-v0.2.Q5_K_M.gguf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1176  100  1176    0     0   2838      0 --:--:-- --:--:-- --:--:--  2840
100 4893M  100 4893M    0     0  37.8M      0  0:02:09  0:02:09 --:--:-- 37.7M13  0:00:57 39.1M
mistral-7b-instruct-v0.2.Q4_0.gguf


In [1]:
from llama_cpp import Llama

enable_gpu = True  # offload LLM layers to the GPU (must fit in memory)

llm = Llama(
    model_path="./mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    n_gpu_layers=-1 if enable_gpu else 0,
    n_ctx=2048,
    verbose=False,
)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

In [2]:
prompt = "¿Puedes escribirme una SPARQL query completa (usando Select, filter, service) para obtener el nombre de los países que tienen una población mayor a 100 millones de habitantes?"

prompt_template=f"<s>[INST] {prompt} [/INST]"

response=llm(prompt=prompt_template, max_tokens=1024, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

response

{'id': 'cmpl-716b586d-2019-4464-af7c-3f4bf67861ca',
 'object': 'text_completion',
 'created': 1705912172,
 'model': './mistral-7b-instruct-v0.2.Q5_K_M.gguf',
 'choices': [{'text': '<s>[INST] ¿Puedes escribirme una SPARQL query completa (usando Select, filter, service) para obtener el nombre de los países que tienen una población mayor a 100 millones de habitantes? [/INST] I\'m assuming you have a SPARQL-enabled dataset about countries and their populations available through a SPARQL endpoint. Here is an example of a query using DBpedia as the data source:\n\n```sparql\nPREFIX dbo: <http://dbpedia.org/ontology/>\nSELECT ?countryName WHERE {\n  SERVICE <https://query.wikidata.org/sparql> {\n    SELECT DISTINCT ?countryWikidataId ?population (STR(?population) AS ?populationStr) WHERE {\n      ?country wdt:P31 dbo:Country .\n      ?country wdt:P627-1 ?capital.\n      OPTIONAL { ?country wdt:P1820 ?population }.\n      FILTER(LANG(?capital) = "en" && STRSTARTS(STR(?population), ">") && REGE

In [3]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

print(torch.cuda.get_device_name(0))
#imprimir version pytorch
print(torch.__version__)


12.1
True
0
NVIDIA GeForce RTX 4080
2.1.2+cu121


# Initializing Dynamic Routes

Similar to the `02-dynamic-routes.ipynb` notebook, we will be initializing some dynamic routes that make use of LLMs for function calling

In [2]:
from datetime import datetime
from zoneinfo import ZoneInfo

from semantic_router import Route
from semantic_router.utils.function_call import get_schema


def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London". Do NOT put the place
        name itself like "rome", or "new york", you must provide
        the IANA format.
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    return now.strftime("%H:%M")


time_schema = get_schema(get_time)
time_schema
time = Route(
    name="get_time",
    utterances=[
        "what is the time in new york city?",
        "what is the time in london?",
        "I live in Rome, what time is it?",
    ],
    function_schema=time_schema,
)

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat, time]

In [3]:
time_schema

{'name': 'get_time',
 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London". Do NOT put the place\n    name itself like "rome", or "new york", you must provide\n    the IANA format.\n:type timezone: str\n:return: The current time in the specified timezone.',
 'signature': '(timezone: str) -> str',
 'output': "<class 'str'>"}

# Encoders

You can use alternative Encoders, however, in this example we want to showcase a fully-local Semantic Router execution, so we are going to use a `HuggingFaceEncoder` with `sentence-transformers/all-MiniLM-L6-v2` (the default) as an embedding model.

In [4]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder()

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# `llama.cpp` LLM

From here, we can go ahead and instantiate our `llama-cpp-python` `llama_cpp.Llama` LLM, and then pass it to the `semantic_router.llms.LlamaCppLLM` wrapper class.

For `llama_cpp.Llama`, there are a couple of parameters you should pay attention to:

- `n_gpu_layers`: how many LLM layers to offload to the GPU (if you want to offload the entire model, pass `-1`, and for CPU execution, pass `0`)
- `n_ctx`: context size, limit the number of tokens that can be passed to the LLM (this is bounded by the model's internal maximum context size, in this case for Mistral-7B-Instruct, 8000 tokens)
- `verbose`: if `False`, silences output from `llama.cpp`

> For other parameter explanation, refer to the `llama-cpp-python` [API Reference](https://llama-cpp-python.readthedocs.io/en/latest/api-reference/)

In [16]:
from semantic_router import RouteLayer

from llama_cpp import Llama
from semantic_router.llms.llamacpp import LlamaCppLLM

enable_gpu = True  # offload LLM layers to the GPU (must fit in memory)

_llm = Llama(
    # model_path="./mistral-7b-instruct-v0.2.Q4_0.gguf",
    model_path="./mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    n_gpu_layers=-1 if enable_gpu else 0,
    n_ctx=2048,
    verbose=False,
)
llm = LlamaCppLLM(name="Mistral-7B-v0.2-Instruct", llm=_llm, max_tokens=None)



rl = RouteLayer(encoder=encoder, routes=routes, llm=llm)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                

In [17]:
rl("how's the weather today?")

RouteChoice(name='chitchat', function_call=None, similarity_score=None, trigger=None)

In [18]:
out = rl("what's the time in New York right now?")
print(out)
get_time(**out.function_call)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

name='get_time' function_call={'timezone': 'America/New_York'} similarity_score=None trigger=None


'18:46'

In [9]:
out = rl("what's the time in Rome right now?")
print(out)
get_time(**out.function_call)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

name='get_time' function_call={'timezone': 'Europe/Rome'} similarity_score=None trigger=None


'00:33'

In [10]:
out = rl("what's the time in Bangkok right now?")
print(out)
get_time(**out.function_call)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

name='get_time' function_call={'timezone': 'Asia/Bangkok'} similarity_score=None trigger=None


'06:34'

In [10]:
out = rl("what's the time in Phuket right now?")
print(out)
get_time(**out.function_call)

from_string grammar:
root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number

name='get_time' function_call={'timezone': 'Asia/Bangkok'} similarity_score=None trigger=None


'23:41'

## Cleanup

Once done, if you'd like to delete the downloaded model you can do so with the following:

```
! rm ./mistral-7b-instruct-v0.2.Q4_0.gguf
```